In [16]:
!pip install numpy pandas tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import numpy as np
import pandas as pd

# Sample dataset
data = pd.read_excel('/content/DATASET-STASIUN (1).xlsx')

df = pd.DataFrame(data)


In [18]:
df

,NAME,STASIUN,ALAMAT,LATITUDE,LONGITUDE,KECAMATAN,POPULASI KECAMATAN
0,STASIUN PORIS,PORIS,Jl. Maulana Hasanudin,NaN,NaN,BATUCEPER,16042.0
1,STASIUN PONDOK CINA,PONDOK CINA,Jl. Stasiun Pondok Cina No. 1 Margonda Depok,NaN,NaN,BEJI,171780.0
2,STASIUN UNIVERSITAS INDONESIA,UNIVERSITAS INDONESIA,Jl. Srengseng Sawah No. 1 Jakarta Selatan,NaN,NaN,BEJI,171780.0
3,STASIUN KRANJI,KRANJI,Jl. Jendral Sudirman No. 1 Bekasi barat Kodya ...,NaN,NaN,BEKASI BARAT,267638.0
4,STASIUN BEKASI TIMUR,BEKASI TIMUR,"Jl. Ir. H. Juanda, Duren Jaya, Bekasi Timur, J...",NaN,NaN,BEKASI TIMUR,272320.0
...,...,...,...,...,...,...,...
90,STASIUN TEBET,TEBET,Jl. Lapangan Ros No. 1 Tebet Jakarta Selatan,NaN,NaN,TEBET,241014.0
91,STASIUN TENJO,TENJO,"Jl. Raya Tenjo, Tenjo, Tenjo, Bogor, Jawa Barat",NaN,NaN,TENJO,73845.0
92,STASIUN JAMBUBARU,JAMBUBARU,"Bojong Pandan, Tunjung Teja, Serang, Banten",NaN,NaN,TUNJUNG TEJA,NaN
93,STASIUN CATANG,CATANG,"Bojong Catang, Tunjung Teja, Serang, Banten",NaN,NaN,TUNJUNG TEJA,NaN


In [19]:
from sklearn.preprocessing import LabelEncoder

# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df['POPULASI KECAMATAN'] = user_encoder.fit_transform(df['POPULASI KECAMATAN'])
df['NAME'] = movie_encoder.fit_transform(df['NAME'])

num_users = len(user_encoder.classes_)
num_movies = len(movie_encoder.classes_)


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten

# Define model architecture
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(num_users, 10)(user_input)
movie_embedding = Embedding(num_movies, 10)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vec, movie_vec])

model = Model(inputs=[user_input, movie_input], outputs=dot_product)
model.compile(loss='mse', optimizer='adam')


In [23]:
# Train the model
POPULASI_KECAMATANs = df['POPULASI KECAMATAN'].values
NAMEs = df['NAME'].values
ratings = df['POPULASI KECAMATAN'].values

model.fit([POPULASI_KECAMATANs, NAMEs], ratings, epochs=10, batch_size=2)


Epoch 1/10
48/48 [==============================] - 1s 3ms/step - loss: 759.9724
Epoch 2/10
48/48 [==============================] - 0s 3ms/step - loss: 759.8419
Epoch 3/10
48/48 [==============================] - 0s 3ms/step - loss: 759.6631
Epoch 4/10
48/48 [==============================] - 0s 3ms/step - loss: 759.3787
Epoch 5/10
48/48 [==============================] - 0s 3ms/step - loss: 758.9274
Epoch 6/10
48/48 [==============================] - 0s 3ms/step - loss: 758.2320
Epoch 7/10
48/48 [==============================] - 0s 4ms/step - loss: 757.2600
Epoch 8/10
48/48 [==============================] - 0s 2ms/step - loss: 755.9564
Epoch 9/10
48/48 [==============================] - 0s 3ms/step - loss: 754.3341
Epoch 10/10
48/48 [==============================] - 0s 2ms/step - loss: 752.4448


In [24]:
# Make predictions
NAME = np.array([0])
POPULASI_KECAMATAN = np.array([1])

prediction = model.predict([NAME, POPULASI_KECAMATAN])
print(f"Predicted rating for Station Name and Population: {prediction[0][0]}")


1/1 [==============================] - 0s 67ms/step
Predicted rating for Station Name and Population: 0.0013758344575762749


In [25]:
# Generate recommendations for a user
NAME = np.array([0])  # User1
POPULASI_KECAMATANs = np.arange(num_movies)  # All movies


NAMEs = np.repeat(NAME, num_movies)
predictions = model.predict([NAMEs, POPULASI_KECAMATANs])

sorted_indices = np.argsort(predictions, axis=0)[::-1].flatten()

top_n = 3
recommended_movies = movie_encoder.inverse_transform(sorted_indices[:top_n])

print(f"Top {top_n} movie recommendations for User1:")
for movie in recommended_movies:
    print(movie)


3/3 [==============================] - 0s 3ms/step
Top 3 movie recommendations for User1:
STASIUN CAWANG
STASIUN MANGGARAI
STASIUN TEBET


In [26]:
# Save the model
model.save('recommender_model.h5')